In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
import re
from nltk.tokenize import word_tokenize
import pandas as pd
import pickle
import time
import pycurl
import re
from io import BytesIO
import json
np.random.seed(1)
tf.set_random_seed(1)

In [2]:
glove_path = "../model/glove/glove.pickle"

In [3]:
with open(glove_path, "rb") as f:
    glove_dict = pickle.load(f)

In [4]:
def embedding(word):
    if word in glove_dict.keys():
        return glove_dict[word]
    else:
        return np.random.normal(size=(200))

In [5]:
max_length = 250
input_size = 200
batch_size = 1

In [7]:
c = pycurl.Curl()
c.setopt(c.URL, "http://101.101.164.175:8501/v1/models/BiLSTM_EN:predict")
c.setopt(pycurl.HTTPHEADER, ['Accept: application/json'])
while True:
    X = input("리뷰를 입력하세요.(exit => 종료)")
    if X == "exit":
        break
    X = [X]
    token_X = [word_tokenize(sentence) for sentence in X]
    batch_X = [[embedding(word) for word in sentence] for sentence in token_X]
    batch_X_padded = np.zeros(shape=(batch_size, max_length, input_size))
    for b in range(batch_size):
        batch_X_padded[b, :len(batch_X[b])] = batch_X[b]
    seq_len_ = [len(x) for x in X]

    data = json.dumps({
        "inputs": {
            "input_X": batch_X_padded.tolist(),
            "dropout_keep_prob": 1.0,
            "seq_len": seq_len_
        }
    })

#         buffer 초기화가 잘 안됨
    buffer = BytesIO()
    
    c.setopt(c.POST, True)
    c.setopt(c.POSTFIELDS, data)
    c.setopt(c.WRITEFUNCTION, buffer.write)

    c.perform()
#         print(c.getinfo(pycurl.RESPONSE_CODE))

    body = buffer.getvalue()
    pred = json.loads(body.decode('utf8'))["outputs"][0]

    print("리뷰 \"{}\" 은 약 {}%의 확률로 긍정 리뷰입니다.".format(X[0], int(pred[0]*100)))

리뷰를 입력하세요.(exit => 종료)hey
리뷰 "hey" 은 약 84%의 확률로 긍정 리뷰입니다.
리뷰를 입력하세요.(exit => 종료)exit
